In [1]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

import os
import pickle

In [2]:
# Load index from file

In [3]:
load_dotenv()

True

In [4]:
loaded_faiss_vs = FAISS.load_local(
    folder_path="./vectordb/faiss/nfcorpus/",
    embeddings=OpenAIEmbeddings())

In [5]:
retriever = loaded_faiss_vs.as_retriever(search_kwargs={'k': 5})

In [6]:
# Define the RAG pipeline

In [7]:
model = 'gpt-3.5-turbo-instruct'
llm = OpenAI(model_name=model, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
template = """Answer the question or Explain the topic given this additional context: {context}
Question: {question}"""

In [9]:
prompt = ChatPromptTemplate.from_template(template)

In [10]:
docs_file_path = './dataset/nfcorpus/documents.pkl'
with open(docs_file_path, 'rb') as file:
    docs = pickle.load(file)
print("Documents loaded successfully.")

Documents loaded successfully.


In [11]:
def format_docs(_docs):
    ls = []
    for doc in _docs:
        if doc.page_content in docs:
            ls.append(docs[doc.page_content]["text"][:800])
    return ls

In [12]:
chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} 
         | prompt 
         | llm 
         | StrOutputParser())

In [13]:
# Run the RAG pipeline

In [14]:
query_file_path = './openai_embeddings/nfcorpus/query_embeddings.pkl'
with open(query_file_path, 'rb') as file:
    loaded_queries = pickle.load(file)
print("Query embeddings loaded successfully.")

Query embeddings loaded successfully.


In [15]:
chain.invoke(loaded_queries["PLAIN-2"]["text"])

'\n\nThe answer is still unclear. While some studies have suggested a potential link between statin use and decreased risk of breast cancer, others have found conflicting results. More research is needed to fully understand the relationship between statins and breast cancer risk.'